<span style="font-size:24pt; color:blue; font-family: 'Times New Roman'">P-box Notebook 
    <span style="color:green;"> for creating AR6 style P-box data </span>
</span>
<br>
<span style="font-size:12pt; color:black; font-family:Georgia, serif;font-style:italic">by Praveen Kumar and Robert Kopp.</span>
<br>
<br>
<span style="font-family: 'Times New Roman'; font-size:16pt; color:black"> 
    This is a notebook that can be used to create and vizualize P-box data from raw <a href="https://doi.org/10.5194/egusphere-2023-14" style="color:blue; text-decoration:underline;">FACTS 1.0</a>
    output. Workflows and associated *.nc files are created as part of this notebook. 
<br>
<br>
<span style="font-family:'Courier New', monospace; font-size:12pt; color:black"> <b>Background::</b> <br>
    In order to represent deep uncertainty, Probability boxes (p-boxes) can be constructed. These represent an envelope encompassing alternative probability distributions for describing the same variables. Within this notebook, we construct four p-boxes: <strong>1e, 1f, 2e,</strong> and <strong>2f</strong>.
    <br><br>
    <strong>P-box 1e</strong> and <strong>2e</strong> use ice-sheet projections from an emulator that accurately captures the output of both the GlacierMIP glacier model intercomparison and the ISMIP6 ice-sheet model intercomparison.
    <br><br>
    <strong>P-box 1e</strong> and <strong>1f</strong> both use multi-model intercomparison exercises 
    (<a href="https://www.nature.com/articles/s41586-021-03302-y" style="color:blue; text-decoration:underline;">ISMIP6</a> and <a href="https://doi.org/10.5194/esd-11-35-2020" style="color:blue; text-decoration:underline;">LARMIP2</a>). 
    These p-boxes represent a medium-to-high level of agreement among the participating models. The IPCC AR6 thus assessed medium confidence in their projections of included processes.
</span>
<br><br><br>
    
    
<span style="font-size:14pt; color:blue"> ==> Code structure (... working on optimizing and removing redundancies to make user friendly) </span>

<ul style="font-size:12pt; color:black"> 
Make sure to set the <b>expFolder</b> to point to your facts exp output folder. This notebook creates the following folders within the current directory. 
<li></li>
<li> 1_workflow  </li>
<li> 3_pbox </li>
<li> 4_confidence_level_files </li>
</ul>


In [1]:
import os 
import fun_pbox_Gen_v1 as fn
#
ssps=["ssp119","ssp126","ssp245","ssp370","ssp585"]
#
#
expFolder="/projects/kopp/facts-experiments/221217"
#
region = 'global'

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Workflow folders</li>
    <li> Quantile files</li>
</ol>  
</span>


In [2]:
def copy_and_convert(source_file, dest_folder, region):
    #shutil.copy2(source_file, dest_folder)
    out_file_path = os.path.join(dest_folder, os.path.basename(source_file).split(".nc")[0] + "_quantiles.nc")
    fn.Samples_to_Quantiles(source_file, out_file_path)

path = fn.create_directory("1_workflow")

skip = {
    'wf3e': ['ssp119', 'ssp370'],
    'wf3f': ['ssp119', 'ssp370'],
    'wf4':  ['ssp119', 'ssp245', 'ssp370']
}

for workflow, file_patterns in fn.WF_file_patterns.items():
    for ssp in ssps:
        if ssp in skip.get(workflow, []):
            continue
        
        ssp_path = os.path.join(path, workflow, ssp)
        os.makedirs(ssp_path, exist_ok=True)
        
        # Copy Component files & convert to Quantiles.
        for file in file_patterns:
            component_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{file}_{region}sl.nc'
            copy_and_convert(component_file_path, ssp_path, region)
        
        # Copy Common files & convert to Quantiles.
        for common_file_item in fn.common_files:
            common_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{common_file_item}_{region}sl.nc'
            copy_and_convert(common_file_path, ssp_path, region)
        
        # Copy Total files & convert to Quantiles.
        total_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.total.workflow.{workflow}.{region}.nc'
        copy_and_convert(total_file_path, ssp_path, region)
        
        # Copy VLM file & convert to Quantiles.  
        if region == 'local':
            vlm_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{fn.local_files}.{region}sl.nc'
            copy_and_convert(vlm_file_path, ssp_path, region)


<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Pbox files and folders</li>
</ol>  
</span>

In [3]:
copy_file_with_pattern = lambda ssp, wf, pattern: fn.copy_filename_with_pattern(f"{os.getcwd()}/1_workflow/{wf}/{ssp}", ssp_path, f"*{pattern}*")
copy_all_wf_files = lambda p, s, d: fn.copy_all_files_from(os.path.join(os.getcwd(), f'1_workflow/wf{p.split("_")[1]}/{s}'), d)
#
#
path = fn.create_directory("2_pbox")
#
for pbox in fn.PB_file_patterns:
    os.makedirs(os.path.join(path, pbox), exist_ok=True)
    for ssp in ssps:
        # ..............................................................................
        if pbox in ['pb_2e', 'pb_2f'] and ssp in['ssp119','ssp245','ssp370']: continue
        # ..............................................................................
        ssp_path = os.path.join(path, pbox,ssp)
        os.makedirs(ssp_path, exist_ok=True)
        #
        # 
        if pbox in ['pb_1e','pb_1f']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            # also copy
            copy_file_with_pattern(ssp, 'wf2e', 'larmip.AIS')
            #
            #
            if pbox in ['pb_1e']:
                #
                # AIS Pbox component
                fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.AIS")
                fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                #
                outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                # total
                copy_file_with_pattern(ssp, 'wf2e', 'total.workflow')
                #
                fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1e");               
                fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      
                outfile=f'{ssp_path}/total-workflow.nc'
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
            #
            #
            if pbox in ['pb_1f']:
                #
                # AIS Pbox component.
                fname1=fn.find_filename_with_pattern(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                #
                outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                # total
                copy_file_with_pattern(ssp, 'wf2f', 'total.workflow')
                #
                fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1f");               
                fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      
                outfile=f'{ssp_path}/total-workflow.nc'
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
        #
        #
        if pbox in ['pb_2e']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            # Also copy
            copy_file_with_pattern(ssp, 'wf1e', 'emulandice.AIS')
            copy_file_with_pattern(ssp, 'wf3e', 'deconto21.AIS')
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_AIS') 
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_GIS')
            copy_file_with_pattern(ssp, 'wf1f', 'ipccar5.glaciers')
            #
            # AIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.AIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
            fname3=fn.find_filename_with_pattern(ssp_path,"deconto21.AIS")
            fname4=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #   
            # GIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.GrIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_GIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   
            #
            # Glaciers Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.glaciers")
            fname2=fn.find_filename_with_pattern(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile = f"{ssp_path}/glaciers-pb{pbox.split('_')[1]}-glaciers-{ssp}_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            # total
            copy_file_with_pattern(ssp, 'wf1e', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf3e', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf4' , 'total.workflow') 
            fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2e");               
            fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1e"); 
            fname3=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf3e"); 
            fname4=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}']; 
            outfile=f'{ssp_path}/total-workflow.nc'
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
        #
        #        
        if pbox in ['pb_2f']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            # Also copy
            copy_file_with_pattern(ssp, 'wf1f', 'ipccar5.icesheets_AIS*');
            copy_file_with_pattern(ssp, 'wf3f', 'deconto21.AIS*')
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_AIS*');
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_GIS')
            #
            # AIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
            fname3=fn.find_filename_with_pattern(ssp_path,"deconto21.AIS")
            fname4=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #   
            # GIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_GIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   
            #
            # total
            copy_file_with_pattern(ssp, 'wf1f', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf3f', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf4' , 'total.workflow') 
            fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1f");               
            fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2f"); 
            fname3=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf3f"); 
            fname4=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}']; 
            outfile=f'{ssp_path}/total-workflow.nc'
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")            
        

In [ ]:
def make_directory(path):
    os.makedirs(path, exist_ok=True)

def copy_files_for_pattern(ssp, wf, patterns, ssp_path):
    for pattern in patterns:
        fn.copy_filename_with_pattern(f"{os.getcwd()}/1_workflow/{wf}/{ssp}", ssp_path, f"*{pattern}*")

def generate_pbox_files(ssp_path, patterns, outfile_pattern, ssp, pbox, component):
    infiles = [f"{ssp_path}/{fn.find_filename_with_pattern(ssp_path, pattern)}" for pattern in patterns]
    outfile = f"{ssp_path}/{outfile_pattern.format(pbox=pbox, ssp=ssp, component=component)}"
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)

base_path = fn.create_directory("2_pbox")

for pbox in fn.PB_file_patterns:
    pbox_path = os.path.join(base_path, pbox)
    make_directory(pbox_path)

    for ssp in ssps:
        if pbox in ['pb_2e', 'pb_2f'] and ssp in ['ssp119', 'ssp245', 'ssp370']:
            continue

        ssp_path = os.path.join(pbox_path, ssp)
        make_directory(ssp_path)

        if pbox == 'pb_1e':
            copy_files_for_pattern(ssp, 'wf2e', ['larmip.AIS', 'total.workflow'], ssp_path)
            generate_pbox_files(ssp_path, ["emulandice.AIS", "larmip.AIS"], "icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc", ssp, pbox, "AIS")
            generate_pbox_files(ssp_path, ["total.workflow.wf1e", "total.workflow.wf2e"], "total-workflow.nc", ssp, pbox, "total")
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
            
        elif pbox == 'pb_1f':
            copy_files_for_pattern(ssp, 'wf2f', ['ipccar5.icesheets_AIS', 'larmip.AIS', 'total.workflow'], ssp_path)
            generate_pbox_files(ssp_path, ["ipccar5.icesheets_AIS", "larmip.AIS"], "icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc", ssp, pbox, "AIS")
            generate_pbox_files(ssp_path, ["total.workflow.wf1f", "total.workflow.wf2f"], "total-workflow.nc", ssp, pbox, "total")
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        elif pbox == 'pb_2e':
            copy_files_for_pattern(ssp, 'wf1e', ['emulandice.AIS', 'deconto21.AIS', 'bamber19.icesheets_AIS', 'bamber19.icesheets_GIS', 'ipccar5.glaciers', 'total.workflow'], ssp_path)
            generate_pbox_files(ssp_path, ["emulandice.AIS", "larmip.AIS", "deconto21.AIS", "bamber19.icesheets_AIS"], "icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc", ssp, pbox, "AIS")
            generate_pbox_files(ssp_path, ["emulandice.GrIS", "bamber19.icesheets_GIS"], "icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_GIS_globalsl.nc", ssp, pbox, "GIS")
            generate_pbox_files(ssp_path, ["emulandice.glaciers", "ipccar5.glaciers"], "glaciers-pb{pbox.split('_')[1]}-glaciers-{ssp}_globalsl.nc", ssp, pbox, "glaciers")
            generate_pbox_files(ssp_path, ["total.workflow.wf1e", "total.workflow.wf2e", "total.workflow.wf3e", "total.workflow.wf4"], "total-workflow.nc", ssp, pbox, "total")
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        elif pbox == 'pb_2f':
            copy_files_for_pattern(ssp, 'wf1f', ['ipccar5.icesheets_AIS', 'deconto21.AIS', 'bamber19.icesheets_AIS', 'bamber19.icesheets_GIS', 'total.workflow'], ssp_path)
            generate_pbox_files(ssp_path, ["ipccar5.icesheets_AIS", "larmip.AIS", "deconto21.AIS", "bamber19.icesheets_AIS"], "icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc", ssp, pbox, "AIS")
            generate_pbox_files(ssp_path, ["GrIS1f.FittedISMIP.GrIS", "bamber19.icesheets_GIS"], "icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_GIS_globalsl.nc", ssp, pbox, "GIS")
            generate_pbox_files(ssp_path, ["total.workflow.wf1f", "total.workflow.wf2f", "total.workflow.wf3f", "total.workflow.wf4"], "total-workflow.nc", ssp, pbox, "total")
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Confidence level files files and folders</li>
</ol>  
</span>

In [4]:
path = fn.create_directory("3_confidence_level_files")
fn.generate_confidence_files(os.getcwd()+'/2_pbox', path)